In [1]:
using Pkg
Pkg.activate("../..")
Pkg.instantiate()

  Activating project at `~/AWID`
Precompiling project...
    471.7 ms  ✓ AWID
  1 dependency successfully precompiled in 1 seconds. 205 already precompiled.


In [2]:
using JLD2
X_train = load("../../data/TF-IDF/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../../data/TF-IDF/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../../data/TF-IDF/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../../data/TF-IDF/imdb_dataset_prepared.jld2", "y_test")
nothing

In [3]:
using AWID.NeuralNetwork, Printf

batch_size = 64
model = Chain(
    Dense(size(X_train, 1), 32, relu),
    Dense(32, 1, sigmoid)
)
train_on_batch, test_loss_and_accuracy = setup_training_functions(
    model=model,
    loss_fn=binary_crossentropy,
    accuracy_fn=accuracy,
    optimizer=Adam(),
    x_test=X_test,
    y_test=y_test,
    batch_size=batch_size,
)

epochs = 5
for epoch in 1:epochs
    epoch_total_loss = 0.0f0
    epoch_total_acc = 0.0f0
    num_processed_batches = 0

    epoch_batches = get_epoch_batches(X_train, y_train, batch_size=batch_size, do_shuffle=true)

    t = @elapsed begin
        for (x_batch, y_batch) in epoch_batches
            batch_loss, batch_acc = train_on_batch(x_batch, y_batch)

            epoch_total_loss += batch_loss
            epoch_total_acc += batch_acc
            num_processed_batches += 1
        end

        train_loss = epoch_total_loss / num_processed_batches
        train_acc = epoch_total_acc / num_processed_batches

        test_loss, test_acc = test_loss_and_accuracy()
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)",
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (17.43s) 	Train: (l: 0.64, a: 0.73) 	Test: (l: 0.58, a: 0.84)
Epoch: 2 (8.08s) 	Train: (l: 0.46, a: 0.91) 	Test: (l: 0.44, a: 0.86)
Epoch: 3 (5.62s) 	Train: (l: 0.30, a: 0.94) 	Test: (l: 0.37, a: 0.87)
Epoch: 4 (6.21s) 	Train: (l: 0.20, a: 0.96) 	Test: (l: 0.33, a: 0.87)
Epoch: 5 (5.63s) 	Train: (l: 0.14, a: 0.98) 	Test: (l: 0.32, a: 0.87)
